# From Malthusian stagnation to sustained economic growth

The Malthus hypothesis states that in pre-industrial times, increases in technological sophistication lead to an increase in population and population density in the short- and long-run, but do not permanently raise income per capita levels. In this assignment, we rely on a model by [Ashraf and Galor (2011)](https://www.aeaweb.org/articles?id=10.1257/aer.101.5.2003) and show that when utility depends only on consumption and the number of children, this hypothesis indeed holds true. Even when we allow technology to be endogenous to the population, this continues to be the case. However, when we extent the model such that utility also depends on the education level of the offspring, sustained economic growth can be achieved. This is because households face a quantity-quality tradeoff, where quantity refers to the number of children and quality refers to the education level of children. Increases in the rate of technological prgoress induce a fertility decline as households optimize their utility by reducing the number of offspring, but increasing their level of education.

**Imports and set magics:**

In [1]:
import numpy as np
import scipy as sp
from scipy import optimize
import sympy as sm
import math
from math import log

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
from modelproject import Modelproject
from modelproject import Modelproject_ext
from modelproject import simulate_malthus, bisect_ss_l, bisect_ss_y, bisect_ss_l1, bisect_ss_y1, bisect_ss_l2, bisect_ss_y2

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')


C:\Users\simon\AppData\Local\Temp\ipykernel_9232\3323755168.py:21: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


# The Malthusian Model: Model Description

We consider the following production function:


$$Y_t = (AX)^\alpha L_t^{1-\alpha}\label{eq1}\tag{1}$$

where

* $Y_t$ is output in period t
* $L_t$ is population in period t (growing at rate n)
* $A$ is technology (fixed)
* $X$ is land (fixed)

Hence, output per capita is defined as:

$$Y_t/L_t = y_t = (AX/L_t)^\alpha\label{eq2}\tag{2}$$

implying decreasing returns to land, i.e., $\frac{\partial y}{\partial L}<0$


We assume that household live for two periods. In period 2, they work and receive income y which they can spend on either conumption (c) or having children (n). Household preferences are as follows:

$$U(c_t, n_t)=(1-\gamma) ln(c_t) + \gamma ln(n_t)\label{eq3}\tag{3}$$

The budget constraint is given by:

$$y_t = c_t + \rho n_t\label{eq4}\tag{4}$$

where $\rho$ is the price of having children relative to consumption

## Solving the Model

### We start by optimizing the household's utility subject to the budget constraint
We start out by defining the model parameters and variables using `sympy`. Furthermore, we write up the utility function, $U(c_t,n_t)$, and define the steady states.

In [2]:
# a. variables
Y = sm.symbols('Y')
A = sm.symbols('A')
L = sm.symbols('L')
y = sm.symbols('y')
X = sm.symbols('X')
U = sm.symbols('U')
c = sm.symbols('c')
n = sm.symbols('n')
lt_1= sm.symbols('L_{t+1}')
lt= sm.symbols('L_t')

# b. parameters
alpha = sm.symbols('alpha')
gamma = sm.symbols('gamma')
rho = sm.symbols('rho')

# c. steady states
pstar = sm.symbols('P^\star')
lstar = sm.symbols('L^\star')
ystar = sm.symbols('y^\star')
ustar = sm.symbols('U^\star')

# d. utility function
U = (1-gamma)*sm.ln(c)+gamma*sm.ln(n)
sm.Eq(ustar,U)

Eq(U^\star, gamma*log(n) + (1 - gamma)*log(c))

***
**Next, we determine how households optimally divide their income between consumption and having offspring**
*** 
    
After doing the preliminary work with defining the variables and parameters as well as the utility function we create an object, $t1$, from the class 'Modelproject' and the household problem using the parameters stated in the class:

In [3]:
t1 = Modelproject() # calls the class
t1.solve() # solves the model
print(t1.c) # displays optimal consumption
print(t1.n) # displays the optimal number of children

30.000631141330853
63.635789895026384


We find that housholds optimally spend fraction $(1-\gamma) = 30.00$ of their income on consumption, and fraction $\gamma/\rho = 63.636$ on offspring. 

### In the next step, we are determining the population and population density in steady state

The population evolves as follows:

$$L_{t+1} = n_t L_t\label{eq5}\tag{5}$$

Inserting the optimality condition for population growth, $n_t$, into the equation yields:

$$L_{t+1} = \frac{\gamma}{\rho}(AX)^\alpha L_t^{1-\alpha}\label{eq6}\tag{6}$$

Using (\ref{eq6}) we solve for the population in steady state by setting $L_{t+1}=L_t=L$ and isolate $L$:

***
**Population and population density in steady state (analytical solution)**
***

In [4]:
# a. define equation (6) in steady state, i.e., L_t+1 = L_t = L
ss = sm.Eq(L, (((gamma/rho)*(A*X)**alpha)*(L**(1-alpha))))

# b. isolate L and print population in steady state
pss = sm.solve(ss, L)[0]
sm.Eq(lstar,pss)

Eq(L^\star, (gamma*(A*X)**alpha/rho)**(1/alpha))

Population density is defined as $P_{t+1}=\frac{L_{t+1}}{X} = \frac{\gamma}{\rho}(A)^\alpha \cdot (XL_t)^{1-\alpha} \Rightarrow P^*=\frac{L^*}{X}$. 

We therefore divide the steady state population with $X$ and write up the population density in steady state, $P^*$:

In [5]:
# a. define population density in steady state
P = pss/X

# b. determine the steady state level of population density
sm.Eq(pstar,P)

Eq(P^\star, (gamma*(A*X)**alpha/rho)**(1/alpha)/X)

***
**Population and population density in steady state (numerical solution)** 
*** 
We now want to find the population number in steady state. We use the bisection algorithm which is a derivative free method of finding the roots of a function, i.e. $x_0\in[a,b]$ such that $f(x_0)=0$. The logic behind the algorithm is such that we start with a big interval that we know contains the desired value, $x_0$. We then reduce this interval successively. If the function, $f$ is continuous and takes on values between $f(a)$ and $f(b)$ at the lower and upper bound respectively, then $f$ will take on values between $f(a)$ and $f(b)$ in the interval $[a,b]$. Hence, if $f(a)<\gamma<f(b)$ then it must be the case that there exists some $c\in[a,b]$ such that $f(c)=\gamma$.

We start off by ensuring that the function value evaluated at each of the two bounds are not the same. If this were not the case then the interval would not contain a root.

The algorithm goes as follows:
1. Set $a_0=a$ and $b_0=b$ and make sure that $f(a_0)$ and $f(b_0)$ have opposite signs. 
2. Compute the function value at the midpoint: $f(m_0)$ where $m_0=\frac{a_0+b_0}{2}$ is the midpoint
3. Determine the sub-intervals:
* If $\text{sign}(f(a_0)) ≠ \text{sign}f(m_0)$ then $a_1=a_0$ and $b_1=m_0$.
* If $\text{sign}(f(m_0)) ≠ \text{sign}f(b_0)$ then $a_1=m_0$ and $b_1=b_0$.
4. Repeat step 2 and 3 until $f(m_n)<\varepsilon$.

In [6]:
# a. upper and lower limit have same sign
def samesign(a, b):
        return a * b > 0
# set tolerance level
tol = 1e-6

# b. create the bisection algorithm
def bisect(func, low, high):

# c. find root of continuous function where f(low) and f(high) have opposite signs
    
    # step 1: Assertain that the function evaluated at the lower and upper bound respectively have different signs
    assert not samesign(func(low), func(high))
    
    # step 2: Compute the mid-point 
    for i in range(100):
        midpoint = (low + high) / 2.0
        fm=func(midpoint)
        # step 3: Determine sub-intervals
        if abs(fm) < tol:
            break
        elif samesign(func(low), func(midpoint)):
            low = midpoint
        else:
            high = midpoint
    return midpoint

# d. define function and the parameter values
def f(L, gamma=0.7, rho=1.1, A=25, X=10, alpha=1/3):
        return L-((gamma/rho)*(A*X)**alpha)*(L**(1-alpha))

# e. run the algorithm with f being the function, the lower bound being 0.1 and the upper bound being 1000
L = bisect(f, 0.1, 1000)
print(f'Steady state level of population for given parameter values is: L*=', L)

# f. find the population density in steady state
print(f'The population density in steady state for given parameter values is {L/10}')

Steady state level of population for given parameter values is: L*= 64.42524364329876
The population density in steady state for given parameter values is 6.442524364329875


Hence, we find that the steady state level of labor is 64.425 and a population density of 6.443.

We can do the same as above using `scipy` and arrive at the same result:

In [7]:
# a. call bisect optimzer from .py file and print population steady state
print(f'Steady state population for given parameter values is: {bisect_ss_l(0.1,500):.3f}')

# b. find the population density in steady state
print(f'The population density in steady state for given parameter values is {bisect_ss_l(0.1,500)/10:.3f}')

Steady state population for given parameter values is: 64.425
The population density in steady state for given parameter values is 6.443


Hence, we find, again that the steady state level of population for given the parameter values is 64.425 and that the population density is 6.443.

## Next, we are going to analyze the impact of an exogenous increase in the technology level $A$ on population and population density in steady state

In [8]:
# a. differentiate population in steady state with respect to A
pss_diff = sm.diff(pss, A)

# b. display differentiated population
pss_diff

(gamma*(A*X)**alpha/rho)**(1/alpha)/A

We can see that an increase in A increases the steady state population.

In [9]:
# a. differentiate population density in steady state with respect to A
P_diff = sm.diff(P,A)

# b. display differentiated population density
P_diff

(gamma*(A*X)**alpha/rho)**(1/alpha)/(A*X)

We can see that an increase in A also increases steady state population density.

## In the following, we create a plot of the Malthus model

In [10]:
simulate_malthus("pop")

interactive(children=(FloatSlider(value=0.7, description='$\\gamma$', max=0.99, step=0.05), FloatSlider(value=…

The plot confirms that population in steady state equals 64.43.

 ## We now determine the level of income per capita in the long-run equilibirum, i.e., the income per capita steady state level

Income per capita evolves as follows:

$$y_{t+1} = (\frac{AX}{L_{t+1}})^\alpha = (\frac{AX}{L_{t}n_t})^\alpha = \frac{y_t}{n_t ^\alpha}\label{eq7}\tag{7}$$

Inserting the optimal number of children into this equation yields:

$$y_{t+1}= (\frac{\rho}{\gamma})^\alpha y_t^{1-\alpha}\label{eq8}\tag{8}$$

***
**Income per capita in steady state (analytical solution)**
***
We find the steady state level of income per capita.
This is done by writing up the function as in section 2.1.2, setting $y_{t+1}=y_t=y$ and solving for $y$:

In [11]:
# a. define equation (8) in steady state, i.e., y_t+1 = y_t = y
ss_y = sm.Eq(y, (((rho/gamma)**alpha)*(y)**(1-alpha)))

# b. isolate y and print income per capita in steady state
yss_y = sm.solve(ss_y, y)[0]
sm.Eq(ystar,yss_y)

Eq(y^\star, ((rho/gamma)**alpha)**(1/alpha))

We notice that in steady state, income per capita does not depend on the level of technology A.

***
**Income per capita in steady state (numerical solution)**
***

In [12]:
print(f'Steady state income per capita for given parameter values is: {bisect_ss_y(0.1,500):.4f}')

Steady state income per capita for given parameter values is: 1.5714


In [13]:
simulate_malthus("inc")

interactive(children=(FloatSlider(value=0.7, description='$\\gamma$', max=0.99, step=0.05), FloatSlider(value=…

The plot confirms that income per capity in steady state equals 1.57.

# First extension of the Malthusian Model: Endogenous technological progress

In this first extension, we test whether the implications of the Malthusian model change when assuming that population positively influences the level of technology.

We assume that technology is defined as follows:

$$ A = A \cdot L_t^\phi\label{eq9}\tag{9} $$

Note that this does not change the optimal levels of consumption and children.

The production function is now given as:

$$ Y_t = (AL_t^\phi X)^\alpha L_t^{1-\alpha} = (AX)^\alpha L_t^{1-\alpha + \phi \alpha}\label{eq10}\tag{10} $$

And income per capita is given as: 

$$ y_t = (AX)^\alpha L_t^{\alpha(\phi-1)}\label{eq11}\tag{11}$$

As before, the law of motion for population is given as:

$$ L_{t+1} = n_t L_t \label{eq12}\tag{12}$$

Remember that the optimal n_t is given as:

$$ n_t^* = \frac{\gamma}{\rho} y_t \label{eq13}\tag{13}$$

Insertin the optimality condition of n_t gives us:

$$L_{t+1} = \frac{\gamma}{\rho} (AX)^\alpha L_t^{\alpha(\phi-1)} L_t = \frac{\gamma}{\rho} (AX)^\alpha L_t^{\alpha(\phi-1)+1}\label{eq14}\tag{14} $$

In [14]:
# define new parameter
phi = sm.symbols('phi')

We solve for the population in steady state by setting $L_{t+1}=L_t=L$ and isolate $L$:

In [15]:
# a. define equation (14) in steady state, i.e., L_t+1 = L_t = L
ss1 = sm.Eq(L, (((gamma/rho)*(A*X)**alpha)*L**(alpha*(phi-1)+1)))

# b. isolate L and print population in steady state
pss1 = sm.solve(ss1, L)[0]
sm.Eq(lstar,pss1)

Eq(L^\star, (gamma*(A*X)**alpha/rho)**(-1/(alpha*(phi - 1))))

Accordingly, population density in steady state is given as:

In [16]:
# a. define population density in steady state
P1 = pss1/X

# b. determine the steady state level of population density
sm.Eq(pstar,P1)

Eq(P^\star, 1/(X*(gamma*(A*X)**alpha/rho)**(1/(alpha*(phi - 1)))))

***
**Population and population density in steady state (numerical solution)** 
*** 
We now want to find the population in steady state. We use the bisection algorithm which is a derivative free method of finding the steady state.

In [17]:
# a. upper and lower limit have same sign
def samesign(a, b):
        return a * b > 0

# b. create the bisection algorithm
def bisect(func, low, high):

# c. find root of continuous function where f(low) and f(high) have opposite signs
    
    # step 1: Assertain that the function evaluated at the lower and upper bound respectively have different signs
    assert not samesign(func(low), func(high))
    
    # step 2: Compute the mid-point 
    for i in range(100):
        midpoint = (low + high) / 2.0
        fm=func(midpoint)
        
        # step 3: Determine sub-intervals
        if abs(fm) < tol:
            break
        elif samesign(func(low), func(midpoint)):
            low = midpoint
        else:
            high = midpoint
    return midpoint


# d. define function and the parameter values
def f(L, gamma=0.7, rho=1.1, A=25, X=10, alpha=1/3, phi=0.1):
        return L-(((gamma/rho)*(A*X)**alpha)*L**(alpha*(phi-1)+1))

# e. run the algorithm with f being the function, the lower bound being 0.1 and the upper bound being 1000
L = bisect(f, 0.1, 1000)
print(f'Steady state level of population for given parameter values is: L*=', L)

# f. find the population density in steady state
print(f'The population density in steady state for given parameter values is {L/10}')

Steady state level of population for given parameter values is: L*= 102.34397903159261
The population density in steady state for given parameter values is 10.23439790315926


Hence, we find that the steady state level of labor is 102.344 and a population density of 10.234.

We can do the same as above using `scipy` and arrive at the same result:

In [18]:
# a. find the population in steady state
print(f'Steady state population for given parameter values is: {bisect_ss_l1(0.1,500):.3f}')

# b. find the population density in steady state
print(f'The population density in steady state for given parameter values is: {bisect_ss_l1(0.1,500)/10:.3f}')

Steady state population for given parameter values is: 102.344
The population density in steady state for given parameter values is: 10.234


## In the following, we plot the dynamics implied by the model

In [19]:
simulate_malthus("pop", "ext1")

interactive(children=(FloatSlider(value=0.7, description='$\\gamma$', max=0.99, step=0.05), FloatSlider(value=…

We see that the population converges to a steady state where the populatoin equals 102.34.

## We now determine the level of income per capita in the long-run equilibirum, i.e. the income per capita steady state level

Remember that income per capita is defined as:

$$ y_t = (AX)^\alpha L_t^{\alpha(\phi-1)}\label{eq15}\tag{15}$$

Income per capita evolves as follows:

$$y_{t+1} = (AX)^\alpha (n_tL_t)^{\alpha(\phi-1)}\label{eq16}\tag{16}$$

Inserting the optimal number of children into this equation yields:

$$y_{t+1}= (AX)^\alpha (\frac{\gamma}{\rho} y_t)^{\alpha(\phi-1)} L_t^{\alpha(\phi-1)} = (AX)^\alpha (\frac{\gamma}{\rho} L_t)^{\alpha(\phi-1)} ( y_t)^{\alpha(\phi-1)} = y_t^{\alpha \phi -\alpha +1} (\frac{\gamma}{\rho})^{\alpha(\phi-1)}\label{eq17}\tag{17}$$

***
**Income per capita in steady state (analytical solution)**
***
We find the steady state level of income per capita.
This is done by writing up the function as in section 2.1.2, setting $y_{t+1}=y_t=y$ and solving for $y$:

In [20]:
# a. define equation (17) in steady state, i.e., y_t+1 = y_t = y
ss1_y = sm.Eq(y, (((rho/gamma)**(alpha*(phi-1))*(y)**(alpha*phi-alpha+1))))

# b. isolate y and print income per capita in steady state
yss1_y = sm.solve(ss1_y, y)[0]
sm.Eq(ystar,yss1_y)

Eq(y^\star, ((rho/gamma)**(alpha*(phi - 1)))**(-1/(alpha*(phi - 1))))

We notice that in steady state, income per capita does still not depend on the level of technology A.

***
**Income per capita in steady state (numerical solution)**
***

In [21]:
print(f'Steady state income per capita for given parameter values is: {bisect_ss_y1(0.1,500):.4f}')

Steady state income per capita for given parameter values is: 0.6364


In [22]:
simulate_malthus("inc", "ext1")

interactive(children=(FloatSlider(value=0.7, description='$\\gamma$', max=0.99, step=0.05), FloatSlider(value=…

The plot confirms that income converges to a steady state level of 0.64.

# Second extension of the Malthusian Model: The demographic transition

Imagine now that households do not only derive utility from the number of children, but also from how well educated the children are (often referred to as quality of children) equivalent to the model described in [Galor (2012)](https://link.springer.com/content/pdf/10.1007/s11698-011-0062-7.pdf). The utility function is therefore as follows:

$$ U(c_t,n_t) = (1-\gamma) ln(c_t) + \gamma [ln(n_t) + \beta ln(h_t)]\label{eq18}\tag{18}$$

where $\beta, \gamma \in [0,1]$ and wehre $h$ is human capital.

Human capital is determined by years of education and the rate of technological progress g:

$$h(e,g)= \frac{e}{e+g}\label{eq19}\tag{19}$$

where $h_{eg}= \frac{\partial h_e}{\partial g} >0$

The budget constraint is now given by:

$$y_t = c_t + (\tau^q + \tau^e e)n_ty_t\label{eq20}\tag{20}$$

In [23]:
# define new parameters
tau_q = sm.symbols('tau_q')
tau_e = sm.symbols('tau_e')
beta = sm.symbols('beta')
g = sm.symbols('g')

## We start by determining the optimal level of education and the optimal number of children 

In [24]:
t2 = Modelproject_ext() # calls the class
t2.solve_ext() # solves the model
print(t2.c) # displays optimal consumption
print(t2.n) # displays optimal number of children
print(t2.e) # displays optimal education level

29.99960298477578
0.9861402276691571
0.5164037017014591


We can see that households decide to have $\frac{\gamma}{\tau_q + (g\tau_e\tau_q)^{0.5}} = 0.986$ children with education level $(\frac{g \tau_q}{\tau_e})^{0.5}=0.516$ .

## In order to find out whether advancements in the rate of technological progress induce a fertility transition, we gauge the impact of an exogenous increase in the rate of technological progress, g, on education, e, and the number of children, n

In [25]:
# a. define n 
n = gamma/(tau_q + (g*tau_e*tau_q)**0.5)

# b. differentiate n with respect to g
n_diff = sm.diff(n, g)

# c. displays the differential with respect to g
n_diff

-0.5*gamma*(g*tau_e*tau_q)**0.5/(g*(tau_q + (g*tau_e*tau_q)**0.5)**2)

We can see that an increase in the rate of technological progress reduces fertility.

In [26]:
# a. define e
e = ((g*tau_q)/tau_e)**0.5

# b. differentiate e with respect to g
e_diff = sm.diff(e,g)

# c. displays differential with respect to g
e_diff

0.5*(g*tau_q/tau_e)**0.5/g

An increase in the rate of technological progress leads to an increase in the education level.

### In the next step, we are determining the population and population density in steady state

The population evolves as follows:

$$L_{t+1} = n_t L_t\label{eq21}\tag{21}$$

Inserting the optimality condition for n_t into the equation yields:

$$L_{t+1} = \frac{\gamma}{\tau_q + (g\tau_e\tau_q)^{0.5}} L_t\label{eq22}\tag{22}$$

***
**Population and population density in steady state (analytical solution)**
***

In [27]:
# a. define equation (22) in steady state, i.e., L_t+1 = L_t = L
ss2 = sm.Eq(L, ((gamma/tau_q + (g*tau_e*tau_q)**0.5)*L))

# b. isolate L and print
pss2 = sm.solve(ss2, L)[0]
sm.Eq(lstar,pss2)

Eq(L^\star, 0.0)

This implies that the population grows at a constant rate as long as it is not equal to zero.

Population density is defined as $P_{t+1}=\frac{L_{t+1}}{X} = \frac{\gamma}{\rho}(A)^\alpha \cdot (XL_t)^{1-\alpha} \Rightarrow P^*=\frac{L^*}{X}$. 

We therefore divide the steady state population with $X$ and write up the population density in steady state, $P^*$:

In [28]:
# a. define popoulation density in steady state
P2 = pss2/X

# b. display population desnity in steady state
sm.Eq(pstar,P2)

Eq(P^\star, 0)

This implies that the population density grows at a constant rate as long as it is not equal to zero.

***
**Population in steady state (numerical solution)**
***

In [29]:
# a. upper and lower limit have same sign
def samesign(a, b):
        return a * b > 0

# b. create the bisection algorithm
def bisect(func, low, high):

# c. Find root of continuous function where f(low) and f(high) have opposite signs
    
    # step 1: Assertain that the function evaluated at the lower and upper bound respectively have different signs
    assert not samesign(func(low), func(high))
    
    # step 2: Compute the mid-point 
    for i in range(100):
        midpoint = (low + high) / 2.0
        fm=func(midpoint)
        
        # step 3: Determine sub-intervals
        if abs(fm) < tol:
            break
        elif samesign(func(low), func(midpoint)):
            low = midpoint
        else:
            high = midpoint
    return midpoint


# d. define function and the parameter values
def f(L, gamma=0.7, tau_q=0.4, g = 0.4, tau_e=0.6):
        return L-((gamma/tau_q + (g*tau_e*tau_q)**0.5)*L)

# e. run the algorithm with f being the function, the lower bound being 0.1 and the upper bound being 1000
L = bisect(f, 0, 1000)
print(f'Steady state level of population for given parameter values is: L*=', L)

# f. find the population density in steady state
print(f'The population density in steady state for given parameter values is {L/10}')

Steady state level of population for given parameter values is: L*= 9.313225746154785e-07
The population density in steady state for given parameter values is 9.313225746154785e-08


Hence, we find that a steady state in population and population density only exists if population is zero.

In [30]:
# a. call bisect function to find population in steady state
print(f'Steady state population for given parameter values is: {bisect_ss_l2(0,500):.3f}')

# b. find the population density in steady state
print(f'The population density in steady state for given parameter values is: {bisect_ss_l2(0,500)/10:.3f}')

Steady state population for given parameter values is: 0.000
The population density in steady state for given parameter values is: 0.000


The solution confirms that the steady state only exists if the population is zero.

In [31]:
simulate_malthus("pop","ext2")

interactive(children=(FloatSlider(value=0.7, description='$\\gamma$', max=0.99, step=0.05), FloatSlider(value=…

The plot confirms that a steady state in population only exists for a non-existent population. If population is positive, it grows at a constant rate.

 ## We now determine the level of income per capita in steady state

Income per capita evolves as follows:

$$y_{t+1} = (\frac{AX}{L_{t+1}})^\alpha = (\frac{AX}{L_{t}n_t})^\alpha = \frac{y_t}{n_t ^\alpha}\label{eq23}\tag{23}$$

Inserting the optimal number of children into this equation yields:

$$y_{t+1} = \frac{y_t}{(\frac{\gamma}{\tau_q + (g \tau_e \tau_q)^{0.5}})^\alpha}\label{eq24}\tag{24}$$

***
**Income per capita in steady state (analytical solution)**
***
We find the steady state level of income per capita.
This is done by writing up the function as in section 2.1.2, setting $y_{t+1}=y_t=y$ and solving for $y$:

In [32]:
# a. define the function (24) in steady state, i.e., y_t+1 = y_t = y
ss_y2 = sm.Eq(y, (y/(gamma/tau_q + (g*tau_e*tau_q)**0.5)**alpha))

# b. isolate income per capita
yss_y2 = sm.solve(ss_y2, y)[0]

# c. display income per capita in steady state
sm.Eq(ystar,yss_y2)

Eq(y^\star, 0.0)

This implies that income per capita grows at a constant rate and does not converge to a steady state as long as it is greater than 0. Hence, valuing children's education does induce sustained economic growth.

***
**Income per capita in steady state (numerical solution)**
***

In [33]:
# a. upper and lower limit have same sign
def samesign(a, b):
        return a * b > 0

# b. create the bisection algorithm
def bisect(func, low, high):

# c. Find root of continuous function where f(low) and f(high) have opposite signs
    
    # step 1: Assertain that the function evaluated at the lower and upper bound respectively have different signs
    assert not samesign(func(low), func(high))
    
    # step 2: Compute the mid-point 
    for i in range(100):
        midpoint = (low + high) / 2.0
        fm=func(midpoint)
        
        # step 3: Determine sub-intervals
        if abs(fm) < tol:
            break
        elif samesign(func(low), func(midpoint)):
            low = midpoint
        else:
            high = midpoint
    return midpoint


# d. define function and the parameter values
def f(y, gamma=0.7, tau_q=0.4, g = 0.4, tau_e=0.6, alpha=1/3):
        return y-(y/(gamma/tau_q + (g*tau_e*tau_q)**0.5)**alpha)

# e. run the algorithm with f being the function, the lower bound being 0.1 and the upper bound being 1000
y = bisect(f, -10, 1000)
print(f'Steady state level of income per capita for given parameter values: y* =', y)

Steady state level of income per capita for given parameter values: y* = -2.9802322387695312e-06


In [34]:
# a. call bisect solver and solve for steady state income
print(f'Steady state income per capita for given parameter values is: {bisect_ss_y2(0,500):.3f}')

Steady state income per capita for given parameter values is: 0.000


We can confirm that a steady state in income per capita only exists if the income per capita is zero.

In [35]:
simulate_malthus("inc","ext2")

interactive(children=(FloatSlider(value=0.7, description='$\\gamma$', max=0.99, step=0.05), FloatSlider(value=…

The plot confirms that a steady state in income per capita only exists if income per capita is zero. For a positive income per capita leve, income grows at a constant rate.

# Conclusion

In this assignment, we analyzed the dynamics of stagnation as defined in the Malthusian model. We calculated the optimal consumption and number of children and determined the steady state population, population density and income per capita. We observed that these steady state levels exist, implying that population, population density, and income per capita converge to a certain level in the long-run. We also showed that an increase in the exogenous level of technological sophistication increases population and population density in the long run, but only has a short-term impact on income per capita. Income per capita in the long run remains unaffected even when technolgy improves.

Adjusting the model such that the level of technology depends on the population size does not change these implycations as population, population density, as well as income per capita still converge to a certain level in the long-run.

We extended the model such that households value not only the number of children, but also their human capital. Since human capital positively depends on the level of education, but negatively depends on the rate of technological progress, this model does not confirm the Malthusian prediction of permanent stagnation. Instead, the model predicts that income per capita as well as population and population density grow at a constant rate. Hence, when parents value both quantity and 'quality' of children, sustained economic growth can be achieved. In fact, increases in the rate of technological progress decrease the optimal number of children and increase the desired education level. This means that the fertility decline during the demographic transition - which historically marks the onset of sustained econocmic growth - can be induced by increases in the rate of technological progress.